In [19]:
from mctnet.actions import locate_peaks
from mctnet.data_loading import _load_point_data
import torchio as tio
import numpy as np
from scipy import spatial
import napari

In [21]:
def estimate_error(peaks, ground_truth_peaks):
    breakpoint()
    return None


def test_different_peak_params(prediction_path, ground_truth_coords, min_distances, min_vals, plot_and_pause=False):
    """Estimate the error of the peaks with different peak-finding parameters.
    
    Args: 
        prediction_path (str): Path to the prediction volume.
        ground_truth_coords (list): List of ground truth coordinates (x, y, z).
        min_distances (list): The list of minimum distances you want to try.
        min_vals (list): The list of minimum values (float) you want to try.
        plot_and_pause (bool): If True, plot the results and pause after each parameter error measurement.
        If False, don't plot and don't pause.

    Returns:
        list: The list of errors.
    """
    if type(min_distances) == int:
        min_distances = [min_distances]
    if type(min_vals) == float:
        min_vals = [min_vals]

    errors = []
    for min_distance in min_distances:
        for min_val in min_vals:
            peaks = locate_peaks(
                prediction_path,
                save=True,
                plot=plot_and_pause,
                peak_min_dist=min_distance,
                peak_min_val=min_val
            )

            errors.append(estimate_error(peaks, ground_truth_coords))

    return errors


In [22]:
# errors_dampieri = test_different_peak_params(
#     prediction_path='/home/jake/projects/mctnet/dataset/fiddler/cropped/test_images_whole/dampieri_male_16-image.nii.prediction.nii.gz',
#     ground_truth_coords=np.loadtxt(
#         './dataset/fiddler/cropped_with_csv_labs/test_labels_whole_10/dampieri_male_16-corneas.csv',
#         delimiter=',',
#         dtype=np.float
#     ).astype(int),
#     min_distances=8,
#     min_vals=0.3,
#     # min_distances=[4, 8, 12],
#     # min_vals=[0.2, 0.3, 0.4, 0.5]
# )
# errors_flammula = test_different_peak_params(
#     prediction_path='/home/jake/projects/mctnet/dataset/fiddler/cropped/test_images_whole/flammula_20190925_male_left-image.prediction.nii.gz',
#     ground_truth_coords=np.loadtxt(
#         '',
#         delimiter=',',
#         dtype=np.float
#     ).astype(int),
#     min_distances=[4, 8, 12],
#     min_vals=[0.2, 0.3, 0.4, 0.5]
# )

In [37]:
# # testing only
# peaks, ground_truth_coords = test_different_peak_params(
#     prediction_path='./dataset/fiddler/cropped/test_images_whole/dampieri_male_16-image.nii.prediction.nii.gz',
#     ground_truth_coords=np.loadtxt(
#         './dataset/fiddler/cropped_with_csv_labs/test_labels_whole_10/dampieri_male_16-corneas.csv',
#         delimiter=',',
#         dtype=np.float
#     ).astype(int),
#     min_distances=8,
#     min_vals=0.3,
# )


Locating peaks...
Saving peaks...


In [49]:
peaks = locate_peaks(
    './dataset/fiddler/cropped/test_images_whole/dampieri_male_16-image.nii.prediction.nii.gz',
    save=True,
    plot=False,
    peak_min_dist=7,
    peak_min_val=0.3,
)

ground_truth_coords=np.loadtxt(
    './dataset/fiddler/cropped_with_csv_labs/test_labels_whole_10/dampieri_male_16-corneas.csv',
    delimiter=',',
    dtype=np.float
).astype(int),

In [140]:

viewr = napari.view_points(ground_truth_coords[0], name='ground truths')
viewr.add_points(peaks, name='peaks')

/home/jake/projects/mctnet/venv/lib/python3.8/site-packages/napari/_vispy/vispy_camera.py:109: RuntimeWarning: divide by zero encountered in true_divide
  zoom = np.min(canvas_size / scale)


<Points layer 'peaks' at 0x7f3824627340>

In [156]:

prediction_coords = peaks

tree = spatial.KDTree(prediction_coords)
closest_dists_1_and_2, closest_neighbours_1_and_2 = tree.query(ground_truth_coords[0], k=2)

closest_dists = [x[0] for x in closest_dists_1_and_2]
second_closest_dists = [x[1] for x in closest_dists_1_and_2]
closest_neighbours = [x[0] for x in closest_neighbours_1_and_2]
second_closest_neighbours = [x[1] for x in closest_neighbours_1_and_2]

is_within_distance = closest_dists < 5
correct_prediction_distances = closest_dists[is_within_distance]

correct_prediction_coord_indx = closest_neighbours[is_within_distance]
all_indxs = np.arange(0, len(prediction_coords))
incorrect_prediction_coord_indx = [x for x in all_indxs if x not in correct_prediction_coord_indx]

correct = prediction_coords[correct_prediction_coord_indx]
false_positives = prediction_coords[incorrect_prediction_coord_indx]

In [157]:
viewr.add_points(correct, name='correctly predicted corneas based on within distance')
viewr.add_points(false_positives, name='false positive predicted corneas based on within distance')

<Points layer 'false positive predicted corneas based on within distance [1]' at 0x7f3826f0de80>

In [159]:
# print summary of evaluation
print('Evaluation summary:')
print('\tPercentage correctly predicted:', len(correct) / len(ground_truth_coords[0]) * 100)
print('\tNumber correctly predicted:', len(correct))
print('\tAverage distance (of correct predictions):', np.mean(correct_prediction_distances))
print('\tMedian distance (of correct predictions):', np.median(correct_prediction_distances))
print('\tStandard deviation (of correct predictions):', np.std(correct_prediction_distances))
print('\tMinimum distance (of correct predictions):', np.min(correct_prediction_distances))
print('\tMaximum distance (of correct predictions):', np.max(correct_prediction_distances))
print('\tNumber missing:', len(ground_truth_coords[0]) - len(correct))
print('\tNumber of false positives:', len(false_positives))
print('\tNumber in ground truth:', len(ground_truth_coords[0]))
print('\tNumber in prediction:', len(prediction_coords))

Evaluation summary:
	Percentage correctly predicted: 0.9511128019783146
	Number correctly predicted: 50
	Average distance (of correct predictions): 3.627145077227001
	Median distance (of correct predictions): 3.7416573867739413
	Standard deviation (of correct predictions): 0.8965593057617154
	Minimum distance (of correct predictions): 1.4142135623730951
	Maximum distance (of correct predictions): 4.898979485566356
	Number missing: 5207
	Number of false positives: 4225
	Number in ground truth: 5257
	Number in prediction: 4274
